In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
from ipywidgets import widgets

In [ ]:
data_dir = "/kaggle/input/horse-vs-human-dataset"
horse_dir = os.path.join(data_dir, "horses")
human_dir = os.path.join(data_dir, "humans")

horse_images = os.listdir(horse_dir)[:5]
human_images = os.listdir(human_dir)[:5]

def display_images(image_paths, title):
    plt.figure(figsize=(10, 5))
    for i, img_path in enumerate(image_paths):
        img = mpimg.imread(img_path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(title)
    plt.show()

display_images([os.path.join(horse_dir, img) for img in horse_images], title = "Horse")
display_images([os.path.join(human_dir, img) for img in human_images], title = "Human")

In [ ]:
print(f"total training horse images: {len(os.listdir(horse_dir))}")
print(f"total training horse images: {len(os.listdir(human_dir))}")

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape = (300,300,3)),
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001), metrics = ['accuracy'])

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir, image_size = (300, 300), batch_size = 32, label_mode = 'binary')

dataset_type = type(train_dataset)
print(f"train_dataset inherits from tf.data.Datset: {issubclass(dataset_type, tf.data.Dataset)}")

In [ ]:
sample_batch = list(train_dataset.take(1))[0]

print(f"sample batch data type: {type(sample_batch)}")
print(f"number of elements: {len(sample_batch)}")

image_batch = sample_batch[0]
label_batch = sample_batch[1]

print(f"image batch shape: {image_batch.shape}")
print(f"label batch shape: {label_batch.shape}")

In [ ]:
print(image_batch[0].numpy())

In [ ]:
print(f"max value: {np.max(image_batch[0].numpy())}")
print(f"min value: {np.min(image_batch[0].numpy())}")

In [ ]:
rescale_layer = tf.keras.layers.Rescaling(scale = 1./255)

In [ ]:
image_scaled = rescale_layer(image_batch[0]).numpy()
print(image_scaled)

In [ ]:
print(f"max value: {np.max(image_scaled)}")
print(f"min value: {np.min(image_scaled)}")

In [ ]:
trained_dataset_scaled = train_dataset.map(lambda image, label: (rescale_layer(image), label))

In [ ]:
sample_batch = list(trained_dataset_scaled.take(1))[0]

image_scaled = sample_batch[0][1].numpy()
# print(image_scaled)

print(f"max value: {np.max(image_scaled)}")
print(f"min value: {np.min(image_scaled)}")

In [ ]:
shuffle_buffer_size = 1000
prefetch_buffer_size = tf.data.AUTOTUNE

train_dataset_final = (trained_dataset_scaled
                    .cache()
                    .shuffle(shuffle_buffer_size)
                    .prefetch(prefetch_buffer_size))

In [ ]:
history = model.fit(
    train_dataset_final,
    epochs = 15,
    verbose = 2
)

In [ ]:
acc = history.history['accuracy']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label = 'Training accuracy')
plt.title('Training Accuracy')
plt.legend(loc=0)
plt.show()

In [ ]:
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.inputs, outputs = successive_outputs)

horse_img_files = [os.path.join(horse_dir, f) for f in horse_images]
human_img_files = [os.path.join(human_dir, f) for f in human_images]
img_path = random.choice(horse_img_files + human_img_files)

img = tf.keras.utils.load_img(img_path, target_size=(300, 300)) 
x = tf.keras.utils.img_to_array(img)
x = x.reshape((1,) + x.shape)

x = rescale_layer(x)

successive_feature_maps = visualization_model.predict(x, verbose=False)


layer_names = [layer.name for layer in model.layers[1:]]

for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:

        n_features = feature_map.shape[-1]  
        size = feature_map.shape[1]
        display_grid = np.zeros((size, size * n_features))
        for i in range(n_features):
            x = feature_map[0, :, :, i]
            x -= x.mean()
            x /= x.std()
            x *= 64
            x += 128
            x = np.clip(x, 0, 255).astype('uint8')
            display_grid[:, i * size : (i + 1) * size] = x

        scale = 20. / n_features
        plt.figure(figsize=(scale * n_features, scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
from io import BytesIO

uploader = widgets.FileUpload(accept = "image/*", multiple = True)
display(uploader)
out = widgets.Output()
display(out)

def file_predict(filename, file, out):
    image = tf.keras.utils.load_img(file, target_size = (300, 300))
    image = tf.keras.utils.img_to_array(image)
    image = rescale_layer(image)
    image = np.expand_dims(image, axis = 0)

    prediction = model.predict(image, verbose = 0)[0][0]

    with out:
        if prediction > 0.5:
            print(filename + " is a human")
        else:
            print(filename + " is a horse")

def on_upload_change(change):
    items = change.new
    for item in items:
        file_jpgdata = BytesIO(item.content)
        file_predict(item.name, file_jpgdata, out)

uploader.observe(on_upload_change, names = 'value')